In [64]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse

In [65]:
df_train = pd.read_csv(r'..\...\treino\r2_ig_train_statements.csv', sep=';')
df_train = df_train.drop(df_train[df_train.Polarity =='other'].index)
df_train.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,É tão bom ir na igreja e sentir a presença de ...,r2_ig,1,no,r2_ig_925,_1156010166719959040,3075,"[('é', 'V'), ('tão', 'ADV'), ('bom', 'ADJ'), (...",[],"{'deus': 1, 'presença': 1, 'igreja': 1, 'culto..."
1,Mano eu admiro muito pessoas que entram pra ig...,r2_ig,1,no,r2_ig_73,_1220808284304789504,2659,"[('mano', 'N'), ('eu', 'PROPESS'), ('admiro', ...",[],"{'igreja': 3, 'mundo': 2, 'pessoas': 1, 'palav..."
2,"Fuma não amigo, vamos pra igreja",r2_ig,1,no,r2_ig_901,_1185186519490932741,2239,"[('fuma', 'V'), ('não', 'ADV'), ('amigo', 'N')...",[],"{'amigo': 1, 'igreja': 1}"
3,"A caminho dá igreja, fecha o final de ano na p...",r2_ig,1,no,r2_ig_1965,_815352911769247745,1454,"[('a', 'PREP'), ('caminho', 'N'), ('dá', 'V'),...",[],"{'igreja': 1, 'caminho': 1, 'senhor': 1, 'ano'..."
4,A Igreja virou palanque e agora tá escancarado...,r2_ig,0,no,r2_ig_35,_1183336439376949248,3606,"[('a', 'ART'), ('igreja', 'N'), ('virou', 'V')...",[],"{'igreja': 1, 'respeito': 1, 'falta': 1, 'pala..."
...,...,...,...,...,...,...,...,...,...,...
5347,Vou na igreja festejar meu protetor e agradece...,r2_ig,1,no,r2_ig_92,_1253416246508306432,1574,"[('vou', 'V'), ('na', 'PREP+ART'), ('igreja', ...",[],"{'batalhas': 1, 'igreja': 1, 'vencedor': 1, 's..."
5348,A Receita Federal descobriu que as igrejas est...,r2_ig,0,no,r2_ig_1949,_1255804576944570369,4264,"[('``', 'N'), ('a', 'ART'), ('receita', 'N'), ...",[],"{'remuneração': 2, 'pastor': 1, 'lucros': 1, '..."
5349,"Com a morte de Mary I, subiu ao trono a filha ...",r2_ig,0,no,r2_ig_618,_1275062697265188864,4665,"[('com', 'PREP'), ('a', 'ART'), ('morte', 'N')...",[],"{'mary': 2, 'i': 2, 'esposa': 1, 'país': 1, 'i..."
5350,"Por isso que eu como católico ""praticante"" (no...",r2_ig,1,no,r2_ig_35,_1042114756046401536,1723,"[('por', 'PREP'), ('isso', 'PROSUB'), ('que', ...",[],"{'igreja': 1, 'tipo': 1, 'feriado': 1, 'nomenc..."


In [66]:
df_test = pd.read_csv(r'..\...\Teste\r2_ig_test_statements.csv', sep=';')
df_test = df_test.drop(df_test[df_test.Polarity =='other'].index)
df_test.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,"Eu sou cristão, apoio o presidente, mas não a ...",r2_ig,0,no,r2_ig_1830,_1242458658644144131,2728,"[('eu', 'PROPESS'), ('sou', 'V'), ('cristão', ...",[],"{'fome': 1, 'privatizações': 1, 'apoio': 1, 'i..."
1,Lá na igreja hj a Yasmin me chamou pra ir lá n...,r2_ig,1,no,r2_ig_972,_1042228960535437313,1557,"[('lá', 'ADV'), ('na', 'PREP+ART'), ('igreja',...",[],"{'pastor': 1, 'igreja': 1, 'hj': 1, 'yasmin': ..."
2,Amo ir na igreja da pastora Rosicleia ❤❤❤,r2_ig,1,no,r2_ig_391,_954181989342564352,1708,"[('amo', 'V'), ('ir', 'V'), ('na', 'PREP+ART')...",[],"{'igreja': 1, 'pastora': 1}"
3,Jesus como eu amo essa igreja ❤️,r2_ig,1,no,r2_ig_384,_1131413214070214656,4285,"[('jesus', 'N'), ('como', 'PREP'), ('eu', 'PRO...",[],"{'igreja': 1, 'jesus': 1}"
4,Nós não podemos fazer nada quanto às pessoas q...,r2_ig,1,no,r2_ig_565,_1228102065358098432,1360,"[('nós', 'PROPESS'), ('não', 'ADV'), ('podemos...",[],"{'pessoas': 1, 'inferno': 1, 'igreja': 1, 'mis..."
...,...,...,...,...,...,...,...,...,...,...
1780,qual o lugar onde tem mais pessoas cadastradas...,r2_ig,0,no,r2_ig_2098,_730440389254090752,702,"[('qual', 'PRO-KS'), ('o', 'ART'), ('lugar', '...",[],"{'pessoas': 1, 'inferno': 1, 'igreja': 1, 'hós..."
1781,Equívoco. Toda Igreja é parcial. E o Estado é ...,r2_ig,0,no,r2_ig_1939,_1070860896476504064,1301,"[('equívoco', 'N'), ('.', 'PU'), ('toda', 'PRO...",[],"{'igreja': 2, 'católicos': 1, 'religiões': 1, ..."
1782,UE? Religião não é amor? Respeito tolerância? ...,r2_ig,0,no,r2_ig_1499,_1183027690632290304,46,"[('ue', 'N'), ('?', 'PU'), ('religião', 'N'), ...",[],"{'dia': 2, 'tolerância': 1, 'cristão': 1, 'igr..."
1783,Bolsonaro abre igrejas por decreto. Alguns Pas...,r2_ig,0,yes,r2_ig_489,_1243148034680205315,4761,"[('bolsonaro', 'N'), ('abre', 'V'), ('igrejas'...",[],"{'coronavírus': 1, 'bolsonaro': 1, 'cultos': 1..."


In [67]:
#carregando unlabeled data
df_u = pd.read_csv(r'..\...\Unlabeled/igreja.csv')
df_u

,Unnamed: 0,Text
0,0,Quem diz que é normal nunca nem entrou em uma ...
1,1,"evangélicos"" até saírem da igreja e virarem a ..."
2,2,"Kkkkkkkkkkkk essa daí e profissional , e da ig..."
3,3,Minha mãe chegou da igreja agora é disse que f...
4,4,Ahahahahaha mds labaxurias é um linguajar estr...
...,...,...
34585,34585,Sou mt princesa do senhor com fotinha no face ...
34586,34586,Quando eu tinha que fzr as coisas na igreja
34587,34587,"Vamos sair"" ""Vamos, mas primeiro vou pra igreja"
34588,34588,Wi-Fi daqui da igreja ta melhor que o la de casa


In [68]:
rows_train = len(df_train.index)
rows_test = len(df_test.index)
rows_label=rows_train+rows_test

In [69]:
rows_unlabel = len(df_u.index)
rows_unlabel

34590

In [70]:
frames = [df_train.Text, df_test.Text, df_u.Text]
X = pd.concat(frames)
X

0        É tão bom ir na igreja e sentir a presença de ...
1        Mano eu admiro muito pessoas que entram pra ig...
2                         Fuma não amigo, vamos pra igreja
3        A caminho dá igreja, fecha o final de ano na p...
4        A Igreja virou palanque e agora tá escancarado...
                               ...                        
34585    Sou mt princesa do senhor com fotinha no face ...
34586          Quando eu tinha que fzr as coisas na igreja
34587      Vamos sair" "Vamos, mas primeiro vou pra igreja
34588     Wi-Fi daqui da igreja ta melhor que o la de casa
34589    Mandei msgn para Yasmin e ela ta na igreja com...
Name: Text, Length: 41727, dtype: object

In [71]:
#vetoriza conjunto completo
from sklearn.feature_extraction.text import TfidfTransformer


In [72]:
from sklearn.feature_extraction.text import TfidfTransformer
vect = TfidfVectorizer(ngram_range=(2, 3),analyzer='char_wb').fit(X)


X_tf = vect.transform(X)
X_tf.shape
X_ori = X_tf
X_ori.shape

(41727, 24078)

In [73]:
x_train = X_ori[0:rows_train,:]
x_test = X_ori[rows_train:rows_train+rows_test,:]
x_test.shape


(1785, 24078)

In [74]:
y_train = df_train.Polarity
y_test = df_test.Polarity
y_test.shape

(1785,)

In [75]:
#classificação
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
model = LogisticRegression(random_state=0)
model.fit(x_train,y_train)
pred_values = model.predict(x_test)     
acc = accuracy_score(pred_values , y_test)
acc

0.7843137254901961

In [76]:
f1_score(y_test, pred_values, average='macro')

0.7843093930281615

In [77]:
precision_score(y_test, pred_values, average='macro')

0.7844724958433587

In [78]:
recall_score(y_test, pred_values, average='macro')

0.784406779661017

In [79]:
#predição dos não-rotulados
X_label = X_ori[rows_label:rows_label+ rows_unlabel]
pred_values_label= model.predict(X_label) 


In [80]:
#avalia a probabilidade de estar certo
prob_label = model.predict_proba(X_label) 


In [81]:
#seleciona os itens que vão entrar no conjunto de treino
threshold = 0.4
t_size = int(threshold*rows_unlabel) #quanto vai adicionar a cada iteração
rows_train = x_train.shape[0]
rows_train

5352

In [82]:
reliable_pred_favor, reliable_pred_against  = [], []
for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
    #se 1, inclui no favor; senão, no against
    #incluir id, probabilidade e valor predito (1 ou 0)
    (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    

In [83]:
#coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)

In [84]:
y= np.asarray(y_train)
index_list=[x for x in range(0, rows_train)] #indices para o novo X a partir do X original
iteractions = 0
position = 0
index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

while iteractions < 0.99:
    iteractions = iteractions + threshold
    for i in range(int(t_size/2)):
        #pega um a favor
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 1) #empilha o y
        index_labels.remove(index+rows_label)
        #pega um contra
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 0) #empilha o y
        index_labels.remove(index+rows_label)
        #anda uma posição nos dois 
        position+=1

    X_train_novo = X_ori[index_list,:]#atualiza o X rotulado com os índices que estão em index_list
    X_label = X_ori[index_labels,:] #pega só os índices que sobraram
    rows_unlabel = X_label.get_shape()[0] #novo número de linhas sem rótulo
    rows_label = X_train_novo.get_shape()[0] + rows_test #linhas com rótulo novas + as de teste que estão guardadas
    X_ori = sparse.vstack([X_train_novo, x_test, X_label]) #empilha treino, teste e unlabelled
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)] #atualiza os unlabelled
    position = 0

    model_novo = LogisticRegression(random_state=0)
    model_novo.fit(X_train_novo,y)
    #testa nos conjuntos originais
    pred_values_novo = model_novo.predict(x_test)     
    acc = accuracy_score(pred_values_novo , y_test)
    f1 = f1_score(y_test, pred_values_novo, average='macro')
    print("iteraction:",iteractions, "accuracy:", acc, "f1_score:", f1)
    
    pred_values_label= model_novo.predict(X_label) 
    prob_label = model.predict_proba(X_label) 
    reliable_pred_favor, reliable_pred_against  = [], []
    for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
        (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    
    reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
    reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)    
    
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

iteraction: 0.4 accuracy: 0.7675070028011205 f1_score: 0.7670847732505693
iteraction: 0.8 accuracy: 0.7714285714285715 f1_score: 0.7703734159334894


IndexError: list index out of range